In [1]:
import pandas as pd
import numpy as np

# Reading and merging data from CVM
CVM is brazilians securities commission. 
On their website (http://dados.cvm.gov.br/dataset/fi-doc-eventual) you can find files that encapsulate all the reports sent by fund managers through the years. The files features are: fund type, employer ID number, fund name, document competence date, date of receipt of the document, link to download the file, file name and document type.

Files from 2015 to 04-26-2021 were appended.

In [2]:
dfcvm = pd.read_excel('../data/portal_data/eventual_fi_2021.xls')
dfcvm = dfcvm.append(pd.read_excel('../data/portal_data/eventual_fi_2020.xls'))
dfcvm = dfcvm.append(pd.read_excel('../data/portal_data/eventual_fi_2019.xls'))
dfcvm = dfcvm.append(pd.read_excel('../data/portal_data/eventual_fi_2018.xls'))
dfcvm = dfcvm.append(pd.read_excel('../data/portal_data/eventual_fi_2017.xls'))
dfcvm = dfcvm.append(pd.read_excel('../data/portal_data/eventual_fi_2016.xls'))
dfcvm = dfcvm.append(pd.read_excel('../data/portal_data/eventual_fi_2015.xls'))

dfcvm = dfcvm.reset_index(drop=True)

Usefull columns

In [3]:
dfcvm=dfcvm[[ 'CNPJ_FUNDO', 'DENOM_SOCIAL', 'DT_COMPTC', 'TP_DOC', 'NM_ARQ', 'LINK_ARQ']]

In [4]:
dfcvm.shape

(325736, 6)

# Reading Economatica data
Economatica is an investment platform. A table was created on their platform with the target values we want to explore.

In [4]:
dfeco=pd.read_csv('../data/economatica_04-26-20.csv', index_col=0 )
dfeco.columns = [i.replace('\n',' ') for i in dfeco.columns]

Usefull columns

In [5]:
dfeco = dfeco[['Name', 'Employer ID number', 'Administrator', 'Series Start Date', 'Res 3792/4661 (acc regul)', 'Date of regulation']]
dfeco.head()

,Name,Employer ID number,Administrator,Series Start Date,Res 3792/4661 (acc regul),Date of regulation
1,051 Acoes FIA,34172417000153,BTG Pactual Servicos Financeiros SA Dtvm,2019-08-22,-,2020-10-27
2,051 Allocation FICFI Mult Cred Priv,36016411000112,BTG Pactual Servicos Financeiros SA Dtvm,2020-06-16,-,2020-07-28
3,051 Alocacao Acoes FIA,36498260000186,BTG Pactual Servicos Financeiros SA Dtvm,2020-11-27,-,2020-12-08
4,051 Alocacao Moder FICFI Mult Cred Priv,36498567000187,BTG Pactual Servicos Financeiros SA Dtvm,2020-06-16,-,2021-01-28
5,051 Blue Marlin Fc FI Mult Cred Priv Ie,3998582000135,Credit Suisse Hedging Griffo Cor Val S.A,2015-03-31,-,2020-09-29


## Cleaning Data
There are many different types of documents, we are only interested on "REGUL FDO"

CNPJ to be cleared from special characters

funds with no file will be useless

Employer ID number also has to be str

In [6]:
dfcvm = dfcvm[dfcvm['TP_DOC']=='REGUL FDO']
dfcvm['CNPJ_FUNDO'] = dfcvm['CNPJ_FUNDO'].str.replace('[^0-9]','')
dfcvm.dropna(subset=['NM_ARQ'], inplace=True)
dfeco['Employer ID number'] = dfeco['Employer ID number'].astype(str)

# Rebalancing
There are many more funds that dont follow the resolution, rebalamcing was needed

Economatica has the resolution status on the present day, not from past reports. Research with specialists showed that funds don't usually change its status (98%).

To decrease the imbalance of our binomial distribution we decided to extend the status of the present report to  past documnets.



In [7]:
cont0, count1, cont_undefinde = dfeco['Res 3792/4661 (acc regul)'].value_counts()

In [8]:
df_class0 = dfeco[dfeco['Res 3792/4661 (acc regul)']=='-']
df_class1 = dfeco[dfeco['Res 3792/4661 (acc regul)']=='Yes']

In [9]:
df_class0_under = df_class0.sample(count1, random_state=42)

In [10]:
dfeco = pd.concat([df_class0_under, df_class1], axis=0)

# Merging
with the premise that an Employer ID that follows Res 4661 has always followed it, the merge will happend only on CNPJ/EmployerID

In [11]:
cvm_and_eco = pd.merge(dfcvm, dfeco, how='inner', left_on=['CNPJ_FUNDO'], right_on=['Employer ID number'])

## Fixing Target
used a dictionary com clean target to ones and zeros

In [12]:
d = dict(zip(cvm_and_eco['Res 3792/4661 (acc regul)'].unique(),[1,0]))

In [13]:
cvm_and_eco['Res 3792/4661 (acc regul)'] = [d[i] for i in cvm_and_eco['Res 3792/4661 (acc regul)']]

## Analyses
Transforming complex files to plain will have total different routes depending on the type of file

In [14]:
doc=0
pdf=0
other=0
for i in list(cvm_and_eco['NM_ARQ']):
    if i[-3:].upper()=='DOC':
        doc+=1
    elif i[-3:].upper()=='PDF':
        pdf+=1
    else:
        print(i[-3:])
        
print('pdf',pdf)
print('doc',doc)
print('other',other)

pdf 16707
doc 2178
other 0


It is expected that files with the sabe Administrator will be simmilar

In [15]:
cvm_and_eco['Administrator'].value_counts()[:15]

Bem Dtvm                                    2496
BNY Mellon Servicos Financeiros Dtvm SA     2473
BTG Pactual Servicos Financeiros SA Dtvm    1419
Intrag                                      1379
Itau Unibanco SA                            1115
Singulare Corretora de Titulos e Valores     881
Santander Caceis Brasil Dtvm SA              796
Bradesco                                     726
-                                            662
Credit Suisse Hedging Griffo Cor Val S.A     659
Safra Servicos de Adm Fiduciaria             579
Banco Santander (Brasil) SA                  569
BB Dtvm S.A                                  529
BNP Paribas                                  505
Brl Dtvm                                     501
Name: Administrator, dtype: int64

# Create a df with one of each Administrator

In [18]:
administrator_df = cvm_and_eco.drop_duplicates(subset=['Administrator'])

In [19]:
administrator_df.to_json('../data/administrator_df.json')

## Now we have a Data Frame with all the info we needed to download the files and and find Target

In [ ]:
cvm_and_eco.to_json('../data/cvm_merge_eco.json')